In [1]:
from functools import cache
from itertools import chain, combinations

In [2]:
leads_to = dict()
rates = dict()
with open("day_16_input.txt") as file:
    while line := file.readline().rstrip():
        sent_1, sent_2 = line.split(";")
        sent_1, sent_2 = sent_1.split(), sent_2.split()
        from_v, v_rate = sent_1[1], int(sent_1[4].split("=")[-1])
        leads_to[from_v] = dict()
        for s in sent_2[4:]:
            leads_to[from_v][s[:2]] = 1
        rates[from_v] = v_rate

In [3]:
for tunnel in leads_to:
    for to_tunnel in leads_to[tunnel]:
        if tunnel not in leads_to[to_tunnel]:
            print("This is a directed graph!")

In [4]:
for tunnel in leads_to:
    for to_tunnel in leads_to[tunnel]:
        if to_tunnel not in leads_to:
            print("Not every tunnel is sufficiently described!")

In [5]:
clogged_tunnels = list()
for tunnel in leads_to:
    if tunnel != "AA" and rates[tunnel] == 0:
        clogged_tunnels.append(tunnel)
        for from_t in leads_to[tunnel]:
            for to_t in leads_to[tunnel]:
                if from_t != to_t:
                    leads_to[from_t][to_t] = min(
                        leads_to[from_t][to_t]
                        if to_t in leads_to[from_t]
                        else float("inf"),
                        leads_to[from_t][tunnel] + leads_to[tunnel][to_t],
                    )

for c_tunnel in clogged_tunnels:
    del leads_to[c_tunnel]
    del rates[c_tunnel]
    for other_tunnel in leads_to:
        if c_tunnel in leads_to[other_tunnel]:
            del leads_to[other_tunnel][c_tunnel]

In [6]:
leads_to

{'LG': {'KS': 3, 'KF': 2},
 'IZ': {'QK': 3, 'KB': 3},
 'AI': {'EK': 3, 'AA': 2, 'TJ': 3, 'PB': 2, 'KB': 3},
 'GU': {'MB': 2, 'KS': 3},
 'YE': {'CU': 2},
 'AA': {'KF': 2, 'AI': 2, 'CJ': 3, 'TJ': 3, 'PB': 2},
 'QK': {'KB': 2, 'PB': 2, 'IZ': 3, 'CJ': 3},
 'EK': {'AI': 3},
 'CJ': {'KS': 2, 'AA': 3, 'KF': 3, 'QK': 3},
 'MB': {'GU': 2},
 'KS': {'TJ': 2, 'LG': 3, 'CJ': 2, 'GU': 3, 'CU': 3},
 'CU': {'YE': 2, 'KS': 3},
 'PB': {'QK': 2, 'TJ': 3, 'AA': 2, 'KF': 3, 'AI': 2},
 'KF': {'AA': 2, 'TJ': 3, 'LG': 2, 'CJ': 3, 'PB': 3},
 'TJ': {'KS': 2, 'KF': 3, 'PB': 3, 'AA': 3, 'AI': 3},
 'KB': {'QK': 2, 'IZ': 3, 'AI': 3}}

In [7]:
rates

{'LG': 8,
 'IZ': 20,
 'AI': 11,
 'GU': 14,
 'YE': 24,
 'AA': 0,
 'QK': 15,
 'EK': 22,
 'CJ': 10,
 'MB': 18,
 'KS': 13,
 'CU': 19,
 'PB': 5,
 'KF': 7,
 'TJ': 3,
 'KB': 21}

In [8]:
# Finding the lengths of paths between any two tunnels
for i in range(len(rates)):
    for tunnel in leads_to:
        for from_t in leads_to[tunnel]:
            for to_t in leads_to[tunnel]:
                if from_t != to_t:
                    leads_to[from_t][to_t] = min(
                        leads_to[from_t][to_t]
                        if to_t in leads_to[from_t]
                        else float("inf"),
                        leads_to[from_t][tunnel] + leads_to[tunnel][to_t],
                    )

#### Task 1

In [9]:
@cache
def solo_optimal_path(start_tunnel, viable_set, remaining_time):
    neighbours = [
        to_tunnel
        for to_tunnel in leads_to[start_tunnel].keys()
        if (to_tunnel in viable_set)
        and (leads_to[start_tunnel][to_tunnel] + 1 <= remaining_time)
    ]
    if not neighbours:
        return 0
    scores = [
        rates[neighbour] * (remaining_time - leads_to[start_tunnel][neighbour] - 1)
        + solo_optimal_path(
            neighbour,
            viable_set - {neighbour},
            remaining_time - leads_to[start_tunnel][neighbour] - 1,
        )
        for neighbour in neighbours
    ]
    return max(scores)

In [10]:
# Task 1 solution
result_1 = solo_optimal_path(
    start_tunnel="AA",
    viable_set=frozenset([key for key in leads_to.keys() if key != "AA"]),
    remaining_time=30,
)
result_1

1724

#### Task 2

In [11]:
# from https://docs.python.org/3/library/itertools.html
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s) + 1))


valves = frozenset([key for key in leads_to.keys() if key != "AA"])
subsets = powerset(valves)

In [12]:
result_2 = result_1
for subset in subsets:
    cur_sub = frozenset(subset)
    result_2 = max(
        solo_optimal_path(start_tunnel="AA", viable_set=cur_sub, remaining_time=26)
        + solo_optimal_path(
            start_tunnel="AA", viable_set=valves - cur_sub, remaining_time=26
        ),
        result_2,
    )

In [13]:
result_2

2283